Function to process the TFRecord and create a csv with the fields - filename, class, xmin, ymin, xmax, ymax, prediction_score

In [2]:
# import modules
import csv, os, sys, io
import tensorflow as tf
import pandas as pd
import numpy as np
sys.path.append('/home/ubuntu/data/tensorflow/my_workspace/camera-trap-detection/data/')
from utils import dataset_util
from PIL import ImageFile
from PIL import Image
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [3]:
# input parameters
filename_list = ['/home/ubuntu/data/tensorflow/my_workspace/training_demo/Predictions/snapshot_serengeti_s01_s06-0-10000.record']
batch_size = 2
score_threshold = 0.5

In [4]:
"""This section contains the helper functions needed for 
decoding TFRecord...
1. decode_record: This function decodes a TFexample with the following features
               'image/filename' - a fixed length feature with the file name
               'image/encoded' - a fixed length feature with image encodings
               'image/format' - a fixed length features with image format
               'image/detection/bbox/xmin' - a varaible name feature with normalized xmin values 
               'image/detection/bbox/xmax' - normalized xmax values
               'image/detection/bbox/ymin' - normalized ymin values
               'image/detection/bbox/ymax' - normalized ymax values
               'image/detection/label' - bounding box labels
               'image/detection/score' - prediction score

"""

def decode_record(serialized_example):
    context_features = {
                        'image/filename': tf.FixedLenFeature([], tf.string),
                        'image/encoded': tf.FixedLenFeature([], tf.string),
                        'image/format': tf.FixedLenFeature([], tf.string),
                        "image/detection/bbox/xmin" : tf.VarLenFeature(tf.float32),
                        "image/detection/bbox/xmax" : tf.VarLenFeature(tf.float32),
                        "image/detection/bbox/ymin" : tf.VarLenFeature(tf.float32),
                        "image/detection/bbox/ymax" : tf.VarLenFeature(tf.float32),
                        "image/detection/label" : tf.VarLenFeature(tf.int64),
                        "image/detection/score" : tf.VarLenFeature(tf.float32)
                    }


    context, sequence = tf.parse_single_sequence_example(serialized=serialized_example,
                                              context_features=context_features,
#                                               sequence_features=sequence_features,
                                              example_name=None,
                                              name=None)

    return ({k: v for k, v in context.items()},{k: v for k, v in sequence.items()})

# Create an iterator to traverse the file
dataset = tf.data.Dataset.from_tensor_slices(filename_list)
dataset = tf.data.TFRecordDataset(dataset)
dataset = dataset.map(lambda x: decode_record(serialized_example=x)).batch(batch_size)

iterator = dataset.make_one_shot_iterator() # create the iterator
batch_data = iterator.get_next()

# Run the session and extract the feature values
with tf.Session() as sess:
    (context, sequence) = sess.run(batch_data)
    img = context['image/encoded']
    filename = context['image/filename']
    # Features added during the detection phase 
    xmin_d = (context['image/detection/bbox/xmin'])
    ymin_d = (context['image/detection/bbox/ymin'])
    xmax_d = (context['image/detection/bbox/xmax'])
    ymax_d = (context['image/detection/bbox/ymax'])
    label_d = (context['image/detection/label'])
    score = (context['image/detection/score'])
    
    file_name = [] # list of mages with no groundtruth
    no_prediction = [] # Storing the image name of the correct predictions
    incorrect_prediction = [] # Storing the image name of the incorrect predictions
    
    # index of the boxes with score greater than the threshold
    index_score_gt_threshold = score[1] > score_threshold # can be checked only once. So moved out of the for loop
    xmins_d, ymins_d, xmaxs_d, ymaxs_d, labels_d, scores = [], [], [], [], [], []
    for rec_i in range(len(img)):
        encoded_jpg_io = io.BytesIO(img[rec_i])
        image = Image.open(encoded_jpg_io)
        width, height = image.size
        
        # index of the image
        index_image_rec_i = xmin_d[0][:, 0] == rec_i
        # index of the boxes with score greater than the threshold
        # index_score_gt_threshold = score[1] > score_threshold
        index_img_gt_thres = list(np.where([a and b for a, b in zip(index_image_rec_i, index_score_gt_threshold)])[0])
        
        xmins_d = xmins_d + list(xmin_d[1][index_img_gt_thres])
        ymins_d = ymins_d + list(ymin_d[1][index_img_gt_thres])
        xmaxs_d = xmaxs_d + list(xmax_d[1][index_img_gt_thres])
        ymaxs_d = ymaxs_d + list(ymax_d[1][index_img_gt_thres])
        labels_d = labels_d + list(label_d[1][index_img_gt_thres])
        scores = scores + list(score[1][index_img_gt_thres])
        file_name = file_name + [filename[rec_i].decode('ascii')]*len(index_img_gt_thres)
        
        if len(index_img_gt_thres)==0:
            no_prediction.append(filename[rec_i].decode('ascii'))

In [4]:
# Create pandas dataframe
df_predictions = pd.DataFrame({'labels':labels_d, 
                               'filename':file_name,
                               'score': scores,
                               'xmin': xmins_d,
                               'ymin': ymins_d,
                               'xmax': xmaxs_d,
                               'ymax': ymaxs_d})
df_predictions = df_predictions.append(pd.DataFrame({'filename':no_prediction}))
# write predictions to csv
# df_predictions.to_csv('/home/ubuntu/data/tensorflow/my_workspace/training_demo/Predictions/snapshot_serengeti_s01_s06-0-10000.csv')

/home/ubuntu/test_env/lib/python3.5/site-packages/pandas/core/frame.py:6211: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


In [39]:
# context['image/detection/bbox/ymin'][0].shape
xmin_test = tf.sparse_tensor_to_dense(
    context['image/detection/bbox/xmin'],
    default_value=0,
    validate_indices=True,
    name=None
)

with tf.Session() as sess:
    print(sess.run(xmin_test))
    print(sess.run(xmin_test[0,5]))
# context['image/detection/bbox/xmin']

[[0.35447586 0.35034874 0.35931507 0.38577777 0.32493898 0.34728947
  0.30087775 0.34311253 0.32428855 0.338793   0.33718154 0.26118416
  0.37068707 0.3337175  0.31924954 0.3565906  0.36697128 0.4031487
  0.34903136 0.34533438 0.35516655 0.35425788 0.30074564 0.32234603
  0.3591262  0.3749118  0.35370883 0.3342785  0.30659792 0.33644125
  0.3722968  0.36111253 0.30157354 0.35065016 0.3808985  0.33131224
  0.33277708 0.32537672 0.3539009  0.34703583 0.33562362 0.37821418
  0.38263106 0.31913966 0.3315553  0.32440192 0.34541464 0.36812073
  0.3518625  0.35086823 0.01591034 0.6654858  0.48245886 0.38707793
  0.67438644 0.02038318 0.49792352 0.02922655 0.67533904 0.6854739
  0.03442344 0.02255642 0.0064411  0.5011709  0.00749242 0.0295031
  0.02598085 0.68445396 0.66680187 0.6707916  0.00942581 0.48725036
  0.0217484  0.66822475 0.6602628 ]
 [0.07371732 0.06931121 0.09127627 0.0820457  0.06865181 0.07116377
  0.08504913 0.07222109 0.075162   0.04551738 0.05854836 0.06489731
  0.06899536 0.

In [19]:
"""Predictor Extractor reads the TFRecords with predictions and outputs the csv file with the predictions """
import tensorflow as tf
import pandas as pd

# tf.enable_eager_execution()

def decode_record(serialized_example):
    """This section contains the helper functions needed for 
    decoding TFRecord...
    1. decode_record: This function decodes a TFexample with the following features
                   'image/filename' - a fixed length feature with the file name
                   'image/encoded' - a fixed length feature with image encodings
                   'image/format' - a fixed length features with image format
                   'image/detection/bbox/xmin' - a varaible name feature with normalized xmin values 
                   'image/detection/bbox/xmax' - normalized xmax values
                   'image/detection/bbox/ymin' - normalized ymin values
                   'image/detection/bbox/ymax' - normalized ymax values
                   'image/detection/label' - bounding box labels
                   'image/detection/score' - prediction score

    """


    context_features = {
                        'image/filename': tf.FixedLenFeature([], tf.string),
                        'image/encoded': tf.FixedLenFeature([], tf.string),
                        'image/format': tf.FixedLenFeature([], tf.string),
                        "image/detection/bbox/xmin" : tf.VarLenFeature(tf.float32),
                        "image/detection/bbox/xmax" : tf.VarLenFeature(tf.float32),
                        "image/detection/bbox/ymin" : tf.VarLenFeature(tf.float32),
                        "image/detection/bbox/ymax" : tf.VarLenFeature(tf.float32),
                        "image/detection/label" : tf.VarLenFeature(tf.int64),
                        "image/detection/score" : tf.VarLenFeature(tf.float32)
                    }


    context, sequence = tf.parse_single_sequence_example(serialized=serialized_example,
                                              context_features=context_features,
#                                               sequence_features=sequence_features,
                                              example_name=None,
                                              name=None)

    return ({k: v for k, v in context.items()},{k: v for k, v in sequence.items()})


tfrecord_path_list = ['/home/ubuntu/data/tensorflow/my_workspace/training_demo/Predictions/snapshot_serengeti_s01_s06-0-10000.record']
batch_size = 512
score_threshold = 0.5
output_csv = '/home/ubuntu/data/tensorflow/my_workspace/training_demo/Predictions/snapshot_serengeti_test.csv'


dataset = tf.data.Dataset.from_tensor_slices(tfrecord_path_list)
dataset = tf.data.TFRecordDataset(dataset)
dataset = dataset.repeat(1)
dataset = dataset.map(lambda x: decode_record(serialized_example=x)).batch(batch_size)

# iterator = dataset.make_one_shot_iterator() # create the iterator
# batch_data = iterator.get_next()

# Run the session and extract the feature values
# with tf.Session() as sess:
    
xmins_d, ymins_d, xmaxs_d, ymaxs_d, labels_d, scores, filenames = [], [], [], [], [], [], []
filenames_without_predictions = []


for i, (context, sequence) in enumerate(dataset):
    batch_shape = context['image/detection/bbox/xmin'].dense_shape
    #context['image/detection/bbox/xmin'][2]

    img = context['image/encoded']
    filename = context['image/filename']
    # Features added during the detection phase 
    xmin_d = tf.sparse_tensor_to_dense(context['image/detection/bbox/xmin'])
    ymin_d = tf.sparse_tensor_to_dense(context['image/detection/bbox/ymin'])
    xmax_d = tf.sparse_tensor_to_dense(context['image/detection/bbox/xmax'])
    ymax_d = tf.sparse_tensor_to_dense(context['image/detection/bbox/ymax'])
    label_d = tf.sparse_tensor_to_dense(context['image/detection/label'])
    score = tf.sparse_tensor_to_dense(context['image/detection/score'])


    for rec_i in range(0, int(batch_shape[0])):
        box_counter = 0
        for box_i in range(0, int(batch_shape[1])):
            if score[rec_i, box_i] < score_threshold:
                continue
            xmins_d.append(xmin_d[rec_i, box_i].numpy())
            ymins_d.append(xmax_d[rec_i, box_i].numpy())
            xmaxs_d.append(ymax_d[rec_i, box_i].numpy())
            ymaxs_d.append(ymax_d[rec_i, box_i].numpy())
            labels_d.append(label_d[rec_i, box_i].numpy())
            scores.append(score[rec_i, box_i].numpy())
            filenames.append(filename[rec_i].numpy().decode('utf-8'))

            box_counter += 1

        if box_counter == 0:
            filenames_without_predictions.append(filename[rec_i].numpy().decode('utf-8'))
    
    print('image: {0} finished'.format(i))

            
 # Create pandas dataframe
df_predictions = pd.DataFrame({'labels':labels_d, 
                               'filename':filenames,
                               'score': scores,
                               'xmin': xmins_d,
                               'ymin': ymins_d,
                               'xmax': xmaxs_d,
                               'ymax': ymaxs_d})

df_predictions = df_predictions.append(pd.DataFrame({'filename':filenames_without_predictions}))
# write predictions to csv
df_predictions.to_csv(output_csv, index=False)


image: 0 finished
image: 1 finished
image: 2 finished
image: 3 finished
image: 4 finished
image: 5 finished
image: 6 finished
image: 7 finished
image: 8 finished
image: 9 finished
image: 10 finished
image: 11 finished
image: 12 finished
image: 13 finished
image: 14 finished
image: 15 finished
image: 16 finished
image: 17 finished
image: 18 finished
image: 19 finished


/home/ubuntu/test_env/lib/python3.5/site-packages/pandas/core/frame.py:6211: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


In [18]:
filename[rec_i].numpy().decode('utf-8')

'S1/E02/E02_R5/S1_E02_R5_PICT0056'

In [10]:
str(filename[rec_i])

"tf.Tensor(b'S1/B04/B04_R1/S1_B04_R1_PICT0009', shape=(), dtype=string)"

In [2]:
%%writefile /home/ubuntu/data/tensorflow/my_workspace/camera-trap-detection/data_prep/predictor_extractor.py
"""Predictor Extractor reads the TFRecords with predictions and outputs the csv file with the predictions """
import tensorflow as tf
import pandas as pd

tf.enable_eager_execution()

def decode_record(serialized_example):
    """This section contains the helper functions needed for 
    decoding TFRecord...
    1. decode_record: This function decodes a TFexample with the following features
                   'image/filename' - a fixed length feature with the file name
                   'image/encoded' - a fixed length feature with image encodings
                   'image/format' - a fixed length features with image format
                   'image/detection/bbox/xmin' - a varaible name feature with normalized xmin values 
                   'image/detection/bbox/xmax' - normalized xmax values
                   'image/detection/bbox/ymin' - normalized ymin values
                   'image/detection/bbox/ymax' - normalized ymax values
                   'image/detection/label' - bounding box labels
                   'image/detection/score' - prediction score

    """


    context_features = {
                        'image/filename': tf.FixedLenFeature([], tf.string),
                        'image/encoded': tf.FixedLenFeature([], tf.string),
                        'image/format': tf.FixedLenFeature([], tf.string),
                        "image/detection/bbox/xmin" : tf.VarLenFeature(tf.float32),
                        "image/detection/bbox/xmax" : tf.VarLenFeature(tf.float32),
                        "image/detection/bbox/ymin" : tf.VarLenFeature(tf.float32),
                        "image/detection/bbox/ymax" : tf.VarLenFeature(tf.float32),
                        "image/detection/label" : tf.VarLenFeature(tf.int64),
                        "image/detection/score" : tf.VarLenFeature(tf.float32)
                    }


    context, sequence = tf.parse_single_sequence_example(serialized=serialized_example,
                                              context_features=context_features,
#                                               sequence_features=sequence_features,
                                              example_name=None,
                                              name=None)

    return ({k: v for k, v in context.items()},{k: v for k, v in sequence.items()})


def predictorExtractor(tfrecord_path_list,
                       output_csv,
                       batch_size=512, 
                       score_threshold=0.5):
    dataset = tf.data.Dataset.from_tensor_slices(tfrecord_path_list)
    dataset = tf.data.TFRecordDataset(dataset)
    dataset = dataset.repeat(1)
    dataset = dataset.map(lambda x: decode_record(serialized_example=x)).batch(batch_size)

    xmins_d, ymins_d, xmaxs_d, ymaxs_d, labels_d, scores, filenames = [], [], [], [], [], [], []
    filenames_without_predictions = []


    for i, (context, sequence) in enumerate(dataset):
        batch_shape = context['image/detection/bbox/xmin'].dense_shape
        #context['image/detection/bbox/xmin'][2]

        img = context['image/encoded']
        filename = context['image/filename']
        # Features added during the detection phase 
        xmin_d = tf.sparse_tensor_to_dense(context['image/detection/bbox/xmin'])
        ymin_d = tf.sparse_tensor_to_dense(context['image/detection/bbox/ymin'])
        xmax_d = tf.sparse_tensor_to_dense(context['image/detection/bbox/xmax'])
        ymax_d = tf.sparse_tensor_to_dense(context['image/detection/bbox/ymax'])
        label_d = tf.sparse_tensor_to_dense(context['image/detection/label'])
        score = tf.sparse_tensor_to_dense(context['image/detection/score'])


        for rec_i in range(0, int(batch_shape[0])):
            box_counter = 0
            for box_i in range(0, int(batch_shape[1])):
                if score[rec_i, box_i] < score_threshold:
                    continue
                xmins_d.append(xmin_d[rec_i, box_i].numpy())
                ymins_d.append(ymin_d[rec_i, box_i].numpy())
                xmaxs_d.append(xmax_d[rec_i, box_i].numpy())
                ymaxs_d.append(ymax_d[rec_i, box_i].numpy())
                labels_d.append(int(label_d[rec_i, box_i].numpy()))
                scores.append(score[rec_i, box_i].numpy())
                filenames.append(filename[rec_i].numpy().decode('utf-8'))

                box_counter += 1

            if box_counter == 0:
                filenames_without_predictions.append(filename[rec_i].numpy().decode('utf-8'))

        print('Batch: {0} finished'.format(i))
    
     # Create pandas dataframe
    df_predictions = pd.DataFrame({'labels':labels_d, 
                                   'filename':filenames,
                                   'score': scores,
                                   'xmin': xmins_d,
                                   'ymin': ymins_d,
                                   'xmax': xmaxs_d,
                                   'ymax': ymaxs_d})

    df_predictions = df_predictions.append(pd.DataFrame({'filename':filenames_without_predictions}))
    df_predictions = df_predictions.round({'score':2})
    # write predictions to csv
    df_predictions.to_csv(output_csv, index=False)



Overwriting /home/ubuntu/data/tensorflow/my_workspace/camera-trap-detection/data_prep/predictor_extractor.py


In [21]:
pwd

'/home/ubuntu/data/tensorflow/my_workspace/camera-trap-detection/data_prep'

**main file**

In [41]:
%%writefile /home/ubuntu/data/tensorflow/my_workspace/camera-trap-detection/predictorExtractor_main.py
"""Main function for extracting TFRecord to a csv. 
Uses the helper modules predictor_extractor.py"""

import argparse

from data_prep.predictor_extractor import predictorExtractor


if __name__ == '__main__':
    parser = argparse.ArgumentParser()
    parser.add_argument(
        "--tfrecord_path_list", nargs='+', type=str, required=True,
        help="Path to TFRecord files")
    parser.add_argument(
        "--output_csv", type=str, required=True,
        help="output csv file"
        )
    parser.add_argument(
        "--batch_size", type=int, default=512,
        help="batch size")
    parser.add_argument(
        "--score_threshold", type=float, default=0.5,
        help="score thresholds to write to csv")



    kwargs = vars(parser.parse_args())

    predictorExtractor(**kwargs)

Overwriting /home/ubuntu/data/tensorflow/my_workspace/camera-trap-detection/predictorExtractor_main.py


In [26]:
%cd /home/ubuntu/data/tensorflow/my_workspace/camera-trap-detection/

/home/ubuntu/data/tensorflow/my_workspace/camera-trap-detection


In [42]:
!python predictorExtractor_main \
--tfrecord_path_list '/home/ubuntu/data/tensorflow/my_workspace/training_demo/Predictions/snapshot_serengeti_s01_s06-0-10000.record' \ 
--output_csv '/home/ubuntu/data/tensorflow/my_workspace/training_demo/Predictions/snapshot_serengeti_test2.csv'

SyntaxError: invalid syntax (<ipython-input-42-2e520146664f>, line 2)

In [29]:
output_csv

'/home/ubuntu/data/tensorflow/my_workspace/training_demo/Predictions/snapshot_serengeti_test.csv'